<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - Lab 3: Testing Queries for REST Server

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

Check SQL Version

In [2]:
import sqlite3
ver = sqlite3.sqlite_version_info
print("SQLite Version {}.{}.{}".format(ver[0], ver[1], ver[2]))

SQLite Version 3.40.1


And then we import our movie database

In [3]:
%sql sqlite:///theaters.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [4]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///theaters.sqlite
Done.


[]

In [13]:
%%sql
SELECT    *
FROM      Theater
LEFT JOIN Performance
USING     (TheaterName)

 * sqlite:///theaters.sqlite
Done.


TheaterName,Capacity,PerformanceId,StartTime,PerformanceDate,IMDBKey
Apollo Theatre,40,None,None,None,None
Artcraft Theater,450,1,10:00,2023-01-15,tt1630029
Artcraft Theater,450,2,13:15,2023-01-01,tt0109830
Artcraft Theater,450,3,17:00,2023-01-23,tt0109830
Booth Theater,750,4,21:20,2023-01-11,tt0102926
Capitol Theater,900,5,19:10,2022-10-06,tt0468569
Capitol Theater,900,6,22:45,2022-10-06,tt0468569
Cliff Theater,40,7,19:35,2022-12-16,tt0468569
Los Angeles Theater,1000,8,14:00,2022-12-17,tt0088847
Los Angeles Theater,1000,9,18:30,2022-12-18,tt0088847


## SQL Query Testing Below

Write SQL code for the following tasks:

+ Give a summary of all tickets for a user, like this:
```
curl -X GET http://localhost:7007/users/alice/tickets
{
    "data": [
        {
            "date": "2021-02-22",
            "startTime": "19:00",
            "theater": "Kino",
            "title": "The Shape of Water",
            "year": 2017,
            "nbrOfTickets": 2
        },
        {
            "date": "2021-02-23",
            "startTime": "19:00",
            "theater": "Skandia",
            "title": "Moonlight",
            "year": 2016,
            "nbrOfTickets": 1
        }
    ]
}
```

In [10]:
%%sql
WITH short_customer_data AS
(
    SELECT      count() AS n_tickets, PerformanceId, CustomerName, UserPassword
    FROM        Ticket
    RIGHT JOIN  Customer
    USING       (username)
    WHERE       username="jacob1576"
    GROUP BY    PerformanceId
)

SELECT      *
FROM        short_customer_data
LEFT JOIN   Performance
USING       (PerformanceId)

 * sqlite:///theaters.sqlite
Done.


n_tickets,PerformanceId,CustomerName,UserPassword,StartTime,PerformanceDate,TheaterName,IMDBKey
2,3,Jacob Krucinski,moVieAccount7428@,17:00,2023-01-23,Artcraft Theater,tt0109830
1,5,Jacob Krucinski,moVieAccount7428@,19:10,2022-10-06,Capitol Theater,tt0468569
1,14,Jacob Krucinski,moVieAccount7428@,09:20,2023-02-04,Garneau Theater,tt0876563
1,16,Jacob Krucinski,moVieAccount7428@,12:30,2049-01-01[,Atlantic,tt1856101
1,17,Jacob Krucinski,moVieAccount7428@,21:00,2022-11-06,Astoria,tt0088247


In [12]:
%%sql
-- All in one statement
SELECT      PerformanceDate, StartTime, TheaterName, MovieTitle, ProductionYear, count() as nbrOfTickets
FROM        Ticket
LEFT JOIN   Customer
USING       (username)
LEFT JOIN   Performance
USING       (PerformanceId)
LEFT JOIN   Movie
USING       (IMDBKey)
WHERE       username="jacob1576"
GROUP BY    PerformanceId

 * sqlite:///theaters.sqlite
Done.


PerformanceDate,StartTime,TheaterName,MovieTitle,ProductionYear,nbrOfTickets
2023-01-23,17:00,Artcraft Theater,Forest Gump,1994,2
2022-10-06,19:10,Capitol Theater,The Dark Night,2008,1
2023-02-04,09:20,Garneau Theater,Ponyo,2008,1
2049-01-01[,12:30,Atlantic,Blade Runner 2049,2017,1
2022-11-06,21:00,Astoria,The Terminator,1984,1


+ Get a user's password

In [5]:
%%sql
SELECT UserPassword
FROM   Customer
WHERE  username="jacob1576"

 * sqlite:///theaters.sqlite
Done.


UserPassword
moVieAccount7428@
